<a href="https://colab.research.google.com/github/guanweijun/GWJ_AI_Learning/blob/main/Day2_PythonBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch, numpy as np   #torch 就是 PyTorch 的顶级模块名，多维数组类 torch.Tensor（在 GPU/CPU 上都能跑）

np_arr = np.arange(6).reshape(2,3)  #arrange（）是Numpy数组
tensor = torch.from_numpy(np_arr) #把NumPy数组np_arr零拷贝地变成 PyTorch 张量，二者共享同一块内存——改 NumPy 值就等于改 Tensor 值，反之亦然

print("NumPy:\n", np_arr)
print("Tensor:\n", tensor)
print("Tensor 形状:", tensor.shape)
print("元素个数:", tensor.numel())


NumPy:
 [[0 1 2]
 [3 4 5]]
Tensor:
 tensor([[0, 1, 2],
        [3, 4, 5]])
Tensor 形状: torch.Size([2, 3])
元素个数: 6


In [5]:

back = tensor.numpy() #把 PyTorch 张量再“变回” NumPy 数组，虽然数据是同一块内存，但 Python 会新建一个“数组外壳”对象。
assert np_arr is back #is 比较的是“是不是同一个 Python 对象”（内存地址相同才返回 True）。因为外壳是新的，所以 np_arr is back 得到 False

AssertionError: 

In [6]:
#用tensor来计算梯度，因为只有这样才能调用出计算图，才能调 backward() 得到梯度
x = torch.tensor(3.0, requires_grad=False)   #把输入特征随便设成 3.0，当作已知常量。还是原来tensor格式
print("x:\n", x)
w = torch.tensor(2.0, requires_grad=True)  #把可训练权重随便设成 2.0，当作初始参数，后面靠梯度下降去更新。还是原来tensor格式
print("w:\n", w)
y_true = torch.tensor(9.0) #没有特殊含义，就是告诉 PyTorch：“把这枚小数 9.0 包成张量，让我后面能直接跟其它张量做运算。”

loss = (w * x - y_true)**2      # 手工 MSE，高维度应该用(w * x - y).pow(2).mean()
loss.backward()     #它把计算图里所有需要梯度的叶子节点（requires_grad=True的，本例只有 w）的梯度一次性算出来，并存到对应的 .grad 属性里。
print("∂loss/∂w =", w.grad.item())   # 期望 2*(w*x-y_true)*x = 2*(6-9)*3 = -18

x:
 tensor(3.)
w:
 tensor(2., requires_grad=True)
∂loss/∂w = -18.0


In [83]:
#线性回归 CPU
#造数据
# 100 个点  y = 5x + 4  + 噪声
X = torch.linspace(0, 10, 100).reshape(-1,1) #在 0 到 10 之间均匀取出 100 个数，先排成 1 维，再改成 100×1 的列向量。-1指不制定行数，自己算
true_w, true_b = 5.0, 4.0
y = true_w * X + true_b + torch.randn(X.size()) #生成一个和 X 同样形状的随机张量

#定义参数
w = torch.randn(1, requires_grad=True) #生成 1 个服从标准正态分布 N(0,1) 的随机数
b = torch.randn(1, requires_grad=True) #生成 1 个服从标准正态分布 N(0,1) 的随机数

import time
start = time.time()

#训练循环（显性写法，便于看懂）
lr = 2.7e-2     # 0.001~0.1 是多年实验的“舒适区”，学习率 0.5 太大，一步迈过“谷底”，每次都在对岸来回蹦，grad越蹦越远。0.1 先跑，稳就留；乱跳减半（0.01），慢就加半（0.3）
for epoch in range(500):
    y_pred = w * X + b
    loss = ((y_pred - y)**2).mean()

    loss.backward()   #计算每次循环的瞬时梯度
    if epoch % 50 == 0:
      print(f"epoch {epoch:3d} w.grad={w.grad.item():.3f}  b.grad={b.grad.item():.3f} ")
    with torch.no_grad():        # 手动更新， 是临时关闭梯度计算的上下文管理器，不会被 autograd 记录到计算图里，因此 .grad 不会被更新。
                      #一句话：更新参数时必须脱离计算图，否则图会无限膨胀且梯度完全不对
        w -= lr * w.grad
        b -= lr * b.grad
        w.grad.zero_(); b.grad.zero_()  #不清零，会和上一轮的梯度累加

    if epoch % 50 == 0:
        print(f"epoch {epoch:3d}  loss={loss:.4f}  w={w.item():.3f}  b={b.item():.3f}")

print("CPU 耗时:", time.time() - start, "s")

epoch   0 w.grad=-366.685  b.grad=-56.593 
epoch   0  loss=1007.8395  w=9.974  b=1.920
epoch  50 w.grad=0.003  b.grad=-0.742 
epoch  50  loss=1.5811  w=5.215  b=2.626
epoch 100 w.grad=0.056  b.grad=-0.370 
epoch 100  loss=1.1786  w=5.107  b=3.334
epoch 150 w.grad=0.028  b.grad=-0.188 
epoch 150  loss=1.0743  w=5.053  b=3.695
epoch 200 w.grad=0.014  b.grad=-0.096 
epoch 200  loss=1.0472  w=5.025  b=3.879
epoch 250 w.grad=0.007  b.grad=-0.049 
epoch 250  loss=1.0402  w=5.011  b=3.972
epoch 300 w.grad=0.004  b.grad=-0.025 
epoch 300  loss=1.0383  w=5.004  b=4.020
epoch 350 w.grad=0.002  b.grad=-0.013 
epoch 350  loss=1.0379  w=5.001  b=4.044
epoch 400 w.grad=0.001  b.grad=-0.006 
epoch 400  loss=1.0377  w=4.999  b=4.057
epoch 450 w.grad=0.001  b.grad=-0.003 
epoch 450  loss=1.0377  w=4.998  b=4.063
CPU 耗时: 0.12009215354919434 s


In [85]:
#线性回归，CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using", device)

X_gpu = X.to(device); y_gpu = y.to(device)
w_gpu = torch.randn(1, requires_grad=True, device=device)
b_gpu = torch.randn(1, device=device)
b_gpu.requires_grad_(True)    #原地打开梯度开关

# 同样循环 500 epoch，计时
import time
start = time.time()


#训练循环
lr = 2.7e-2
for epoch in range(501):
    y_pred = w_gpu * X_gpu + b_gpu
    loss = ((y_pred - y_gpu)**2).mean()

    loss.backward()
    if epoch % 50 == 0:
      print(f"epoch {epoch:3d} w_gpu.grad={w_gpu.grad.item():.3f}  b_gpu.grad={b_gpu.grad.item():.3f} ")
    with torch.no_grad():
        w_gpu -= lr * w_gpu.grad
        b_gpu -= lr * b_gpu.grad
        w_gpu.grad.zero_(); b_gpu.grad.zero_()

    if epoch % 50 == 0:
        print(f"epoch {epoch:3d}  loss={loss:.4f}  w_gpu={w_gpu.item():.3f}  b_gpu={b_gpu.item():.3f}")


print("GPU 耗时:", time.time() - start, "s")

Using cpu
epoch   0 w_gpu.grad=-311.562  b_gpu.grad=-47.725 
epoch   0  loss=726.8898  w_gpu=9.120  b_gpu=2.943
epoch  50 w_gpu.grad=-0.025  b_gpu.grad=-0.451 
epoch  50  loss=1.2351  w_gpu=5.129  b_gpu=3.200
epoch 100 w_gpu.grad=0.034  b_gpu.grad=-0.223 
epoch 100  loss=1.0889  w_gpu=5.063  b_gpu=3.626
epoch 150 w_gpu.grad=0.017  b_gpu.grad=-0.114 
epoch 150  loss=1.0510  w_gpu=5.031  b_gpu=3.844
epoch 200 w_gpu.grad=0.009  b_gpu.grad=-0.058 
epoch 200  loss=1.0411  w_gpu=5.014  b_gpu=3.955
epoch 250 w_gpu.grad=0.004  b_gpu.grad=-0.029 
epoch 250  loss=1.0386  w_gpu=5.006  b_gpu=4.011
epoch 300 w_gpu.grad=0.002  b_gpu.grad=-0.015 
epoch 300  loss=1.0379  w_gpu=5.001  b_gpu=4.040
epoch 350 w_gpu.grad=0.001  b_gpu.grad=-0.008 
epoch 350  loss=1.0378  w_gpu=4.999  b_gpu=4.054
epoch 400 w_gpu.grad=0.001  b_gpu.grad=-0.004 
epoch 400  loss=1.0377  w_gpu=4.998  b_gpu=4.062
epoch 450 w_gpu.grad=0.000  b_gpu.grad=-0.002 
epoch 450  loss=1.0377  w_gpu=4.997  b_gpu=4.066
epoch 500 w_gpu.grad=0.

In [43]:
#定义模型, 用class
import torch
import torch.nn as nn

class LinearReg(nn.Module):             #我要开一家叫 LinearReg 的公司，并且挂靠在巨头集团 nn.Module 旗下（继承）
    def __init__(self):             #公司开张第一天，得先把“营业执照”办下来——这就是初始化仪式。在 __init__ 里，self 是空壳对象，你给它装属性、贴方法，把它“装饰成”真正的实例。
        super().__init__()          #先去总部（nn.Module）把营业执照盖章（父类初始化），否则你不能享受集团的“水电优惠”（自动求导、保存、搬家等福利）
        self.linear = nn.Linear(1, 1)   # 单特征→单输出。公司正式进货：买一台官方现成的“线性机”（nn.Linear），它能做 y = w·x + b, linear挂在 self 身上当属性用

    def forward(self, x):           #定义前向传播路径：数据 x 进入模型后，直接传给 self.linear 这一层（即官方线性模块 nn.Linear(1,1)），得到输出 y = w·x + b，无需手动写 w*x + b
        return self.linear(x)       #x这里只是形式变量，和forward里面一致就行

my_model = LinearReg().to(device)         #在外面要叫LinearReg, self只有在上面类的方法内部才存在，代表“当前这个实例”。self 是“屋里照镜子”，LinearReg() 是“屋外生小孩”my_model

#损失 & 优化器
criterion = nn.MSELoss()                    #选损失函数：均方误差 (Mean-Squared-Error)
#optimizer = torch.optim.SGD(my_model.parameters(), lr=2.7e-2)     #选优化器：随机梯度下降 (Stochastic Gradient Descent),把模型里所有需要训练的张量（weight、bias）一次性交给优化器


optimizer = torch.optim.SGD([       #给不同参数组指定不同 lr
    {'params': my_model.linear.weight, 'lr': 1e-2},
    {'params': my_model.linear.bias,   'lr': 1.5e-2}
], lr=1e-3)          # 默认 lr，对未显式分组的生效


#训练 500 epoch（模板化 6 步）
for epoch in range(500):
    optimizer.zero_grad()
    outputs = my_model(X_gpu)
    loss = criterion(outputs, y_gpu)
    loss.backward()           #反向传播，计算 loss 对 所有参数（w、b）的梯度，存到对应的 .grad 里。
    optimizer.step()          #按梯度和学习率更新一次参数：w ← w − lr·grad_w，b ← b − lr·grad_b

    if epoch % 50 == 0:
        w_val = my_model.linear.weight.item()   # 取出标量 w
        b_val = my_model.linear.bias.item()     # 取出标量 b
        print(f'epoch {epoch:d}  loss={loss.item():.4f}  w={w_val:.3f}  b={b_val:.3f}')

epoch 0  loss=780.4953  w=4.007  b=0.774
epoch 50  loss=2.2802  w=5.293  b=2.059
epoch 100  loss=1.6577  w=5.193  b=2.724
epoch 150  loss=1.3605  w=5.124  b=3.183
epoch 200  loss=1.2185  w=5.076  b=3.500
epoch 250  loss=1.1508  w=5.043  b=3.719
epoch 300  loss=1.1184  w=5.020  b=3.871
epoch 350  loss=1.1029  w=5.004  b=3.976
epoch 400  loss=1.0956  w=4.993  b=4.048
epoch 450  loss=1.0920  w=4.986  b=4.098


In [76]:
# 两个特征的模型，用class
# 1. 造数据 -----------------------------------------------------------
# 真实模型: y = 3*x1 + 5*x2 + 4
true_w1, true_w2, true_b = 3.0, 5.0, 4.0
n_samples = 100
X_cpu = torch.rand(n_samples, 2) * 10          # 100×2 矩阵
y_cpu = true_w1 * X_cpu[:, 0] + true_w2 * X_cpu[:, 1] + true_b
y_cpu += torch.randn(n_samples) * 0.5          # 加噪声
X2D_gpu = X_cpu.to(device)  #小心，上面多个celll里面是1个特征，这里2个，所以改个名字
y2D_gpu = y_cpu.to(device)

# 2. 定义模型 ---------------------------------------------------------
class LinearReg2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)   # 2 个特征 → 1 个输出

    def forward(self, x):           #x这里只是形式变量
        return self.linear(x)           # y = w1*x1 + w2*x2 + b

model = LinearReg2D().to(device)

# 3. 损失 & 优化器 -----------------------------------------------------
criterion = nn.MSELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1.5e-2)

optimizer = torch.optim.SGD([ {'params': model.linear.weight, 'lr': 1.5e-2},
    {'params': model.linear.bias,   'lr': 2.5e-2}],
    lr=1.5e-2)          # 默认 lr，对未显式分组的生效


# 4. 训练 -------------------------------------------------------------
for epoch in range(600):
    optimizer.zero_grad()
    outputs = model(X2D_gpu)              # 前向
    loss = criterion(outputs.squeeze(), y2D_gpu)  # squeeze 保证形状一致
    loss.backward()
    optimizer.step()

    if epoch % 60 == 0:
        w1, w2 = model.linear.weight[0].tolist()  # 取出 w1, w2
        b = model.linear.bias.item()              # 取出 b
        print(f'epoch {epoch:3d}  loss={loss.item():.4f}  '
              f'w1={w1:.3f}  w2={w2:.3f}  b={b:.3f}')

epoch   0  loss=2340.5554  w1=7.944  w2=8.644  b=1.759
epoch  60  loss=22.9899  w1=3.546  w2=5.593  b=1.946
epoch 120  loss=0.7581  w1=3.143  w2=5.167  b=2.604
epoch 180  loss=0.3806  w1=3.074  w2=5.086  b=3.089
epoch 240  loss=0.3054  w1=3.048  w2=5.053  b=3.412
epoch 300  loss=0.2739  w1=3.033  w2=5.033  b=3.624
epoch 360  loss=0.2605  w1=3.023  w2=5.020  b=3.762
epoch 420  loss=0.2547  w1=3.016  w2=5.011  b=3.853
epoch 480  loss=0.2522  w1=3.012  w2=5.006  b=3.913
epoch 540  loss=0.2512  w1=3.009  w2=5.002  b=3.952
